In [52]:
import pyodbc
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.utils import to_categorical

In [2]:
class SQLConnection:
    """ SQL Connection Class

        Object that handles connection and querying of a database
    """

    def __init__(self, host, database):
        self.conn = pyodbc.connect(driver='{SQL Server}', server=host, database=database, trusted_connection='yes')

    def query_to_cursor(self, query_string):
        cursor = self.conn.cursor()
        cursor.execute(query_string)
        return cursor
    
    def query_to_df(self, query_string, index):
        df = pd.read_sql(query_string, conn.conn, index_col=index)
        return df

    def end(self):
        self.conn.close()

In [61]:
QUERY = 'SELECT * FROM gestures WHERE record BETWEEN %d AND %d;'

conn = SQLConnection(host='localhost', database='EMG_Gestures')

QUERY_SIZE = 'SELECT COUNT(record) FROM gestures;'

# dataframe = conn.query_to_df(QUERY_SIZE, '')

cursor_size = conn.query_to_cursor(QUERY_SIZE)
data_size = cursor_size.fetchval()

batch_number = 256

batch_size = data_size / batch_number

# a = 0
# b = 4096

# data = (a,b)

#data = conn.query_to_df(QUERY % data, 'record')


In [68]:
import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping



#batch_size = 4096
#batch_number = 256
num_classes = 8
epochs = 20



model = Sequential()

#get number of columns in training data
#n_cols = train_X.shape[1]

model.add(Dense(10, activation='relu', input_shape=(8,)))
model.add(Dropout(0.1))
model.add(Dense(10, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='softmax'))

#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

model.summary()

for i in range(round(batch_number*0.6)):
    a = i*batch_size 
    b = (i+1)*batch_size 
    
    data_range = (a,b)
    
    data = conn.query_to_df(QUERY % data_range, 'record')
    
    data = data.abs()
    
    # Check head rows
    #print(data.head())
    
    # drop class from data for training
    train_X = data.drop(columns=['class'])
    
    #train_X = MinMaxScaler()
    rows = len(train_X)
    cols = len(train_X.columns)
    
    pre_scl = train_X.values.reshape(rows*cols, 1)
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(pre_scl)
    train_X_res = scaler.transform(train_X)
    
    train_X = train_X_res.reshape(rows, cols)
    
    train_X = pd.DataFrame(train_X)
    
    # Check head rows
    train_X.head()
    
    # convert to binarise 7 classes from 'class' column
    train_y = to_categorical(data['class'], num_classes = 8)
    
    # Check head rows after conversion
    train_y[0:5]

    
    model.fit(train_X, train_y, epochs=30, validation_split=0.2, callbacks=[early_stopping_monitor])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 10)                90        
_________________________________________________________________
dropout_23 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                110       
_________________________________________________________________
dropout_24 (Dropout)         (None, 10)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 8)                 88        
Total params: 288
Trainable params: 288
Non-trainable params: 0
_________________________________________________________________
Train on 3276 samples, validate on 819 samples
Epoch 1/30
3276/3276 [==============================]3276/3276 [==============================] -

3277/3277 [==============================]3277/3277 [==============================] - 1s 259us/step - loss: 1.3519 - acc: 0.4739 - val_loss: 1.2388 - val_acc: 0.5476

Epoch 11/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 220us/step - loss: 1.3466 - acc: 0.4739 - val_loss: 1.2330 - val_acc: 0.5549

Epoch 12/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 133us/step - loss: 1.3465 - acc: 0.4687 - val_loss: 1.2362 - val_acc: 0.5488

Epoch 13/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 124us/step - loss: 1.3355 - acc: 0.4797 - val_loss: 1.2349 - val_acc: 0.5634

Epoch 14/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 152us/step - loss: 1.3288 - acc: 0.4825 - val_loss: 1.2196 - val_acc: 0.5634

Epoch 15/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 249us/step


Epoch 9/30
3277/3277 [==============================]3277/3277 [==============================] - ETA: 0s - loss: 1.2509 - acc: 0.523 - 1s 243us/step - loss: 1.2474 - acc: 0.5261 - val_loss: 1.2037 - val_acc: 0.5341

Epoch 10/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 198us/step - loss: 1.2367 - acc: 0.5313 - val_loss: 1.1903 - val_acc: 0.5476

Epoch 11/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 171us/step - loss: 1.2355 - acc: 0.5386 - val_loss: 1.1999 - val_acc: 0.5415

Train on 3277 samples, validate on 820 samples
Epoch 1/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 130us/step - loss: 1.2559 - acc: 0.5114 - val_loss: 1.1230 - val_acc: 0.6000

Epoch 2/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 167us/step - loss: 1.2448 - acc: 0.5133 - val_loss: 1.1275 - val_acc: 0.6122

Epoch 3/30
3277/3


Epoch 4/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 132us/step - loss: 1.1855 - acc: 0.5450 - val_loss: 1.0837 - val_acc: 0.5976

Epoch 5/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 209us/step - loss: 1.1995 - acc: 0.5301 - val_loss: 1.0770 - val_acc: 0.5902

Epoch 6/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 218us/step - loss: 1.1844 - acc: 0.5386 - val_loss: 1.0785 - val_acc: 0.6073

Epoch 7/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 238us/step - loss: 1.1919 - acc: 0.5352 - val_loss: 1.0955 - val_acc: 0.6000

Epoch 8/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 135us/step - loss: 1.1946 - acc: 0.5374 - val_loss: 1.0794 - val_acc: 0.6000

Train on 3277 samples, validate on 820 samples
Epoch 1/30
3277/3277 [==============================]3277

3277/3277 [==============================]3277/3277 [==============================] - 1s 209us/step - loss: 1.1704 - acc: 0.5343 - val_loss: 1.0860 - val_acc: 0.5841

Train on 3277 samples, validate on 820 samples
Epoch 1/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 149us/step - loss: 1.1931 - acc: 0.5331 - val_loss: 1.0858 - val_acc: 0.5915

Epoch 2/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 149us/step - loss: 1.1877 - acc: 0.5392 - val_loss: 1.0832 - val_acc: 0.5829

Epoch 3/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 239us/step - loss: 1.1892 - acc: 0.5331 - val_loss: 1.0803 - val_acc: 0.5854

Epoch 4/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 235us/step - loss: 1.1706 - acc: 0.5511 - val_loss: 1.0791 - val_acc: 0.5890

Epoch 5/30
3277/3277 [==============================]3277/3277 [=====

...

3277/3277 [==============================]3277/3277 [==============================] - 0s 127us/step - loss: 1.1416 - acc: 0.5612 - val_loss: 1.0278 - val_acc: 0.5951

Epoch 3/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 215us/step - loss: 1.1217 - acc: 0.5679 - val_loss: 1.0227 - val_acc: 0.5976

Epoch 4/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 218us/step - loss: 1.1206 - acc: 0.5725 - val_loss: 1.0196 - val_acc: 0.5866

Epoch 5/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 213us/step - loss: 1.1266 - acc: 0.5667 - val_loss: 1.0300 - val_acc: 0.6037 ETA: 0s - loss: 1.1221 - acc: 0.

Epoch 6/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 137us/step - loss: 1.1321 - acc: 0.5688 - val_loss: 1.0333 - val_acc: 0.6037

Epoch 7/30
3277/3277 [==============================]3277/3277 [===================


Epoch 4/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 126us/step - loss: 1.1482 - acc: 0.5652 - val_loss: 1.0662 - val_acc: 0.5829

Train on 3277 samples, validate on 820 samples
Epoch 1/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 175us/step - loss: 1.1132 - acc: 0.5682 - val_loss: 1.0704 - val_acc: 0.6110

Epoch 2/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 214us/step - loss: 1.1231 - acc: 0.5645 - val_loss: 1.0522 - val_acc: 0.6085

Epoch 3/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 254us/step - loss: 1.0961 - acc: 0.5752 - val_loss: 1.0598 - val_acc: 0.6061

Epoch 4/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 149us/step - loss: 1.1170 - acc: 0.5652 - val_loss: 1.0569 - val_acc: 0.6037

Epoch 5/30
3277/3277 [==============================]3277

3277/3277 [==============================]3277/3277 [==============================] - 1s 235us/step - loss: 1.1365 - acc: 0.5609 - val_loss: 0.9791 - val_acc: 0.6280

Epoch 3/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 223us/step - loss: 1.1312 - acc: 0.5661 - val_loss: 0.9861 - val_acc: 0.6232

Epoch 4/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 244us/step - loss: 1.1307 - acc: 0.5667 - val_loss: 0.9924 - val_acc: 0.6134

Epoch 5/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 205us/step - loss: 1.1323 - acc: 0.5612 - val_loss: 0.9885 - val_acc: 0.6195

Train on 3277 samples, validate on 820 samples
Epoch 1/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 245us/step - loss: 1.1056 - acc: 0.5697 - val_loss: 1.0143 - val_acc: 0.5988

Epoch 2/30
3277/3277 [==============================]3277/3277 [=====


Train on 3277 samples, validate on 820 samples
Epoch 1/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 239us/step - loss: 1.1311 - acc: 0.5746 - val_loss: 1.0668 - val_acc: 0.5890

Epoch 2/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 129us/step - loss: 1.1211 - acc: 0.5719 - val_loss: 1.0620 - val_acc: 0.5976

Epoch 3/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 131us/step - loss: 1.1480 - acc: 0.5652 - val_loss: 1.0838 - val_acc: 0.5927

Epoch 4/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 239us/step - loss: 1.1309 - acc: 0.5777 - val_loss: 1.0848 - val_acc: 0.5854

Epoch 5/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 220us/step - loss: 1.1231 - acc: 0.5771 - val_loss: 1.0648 - val_acc: 0.6012

Train on 3277 samples, validate on 820 samples
Epoch 1/30


Epoch 4/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 204us/step - loss: 1.1253 - acc: 0.5713 - val_loss: 1.0501 - val_acc: 0.5890

Epoch 5/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 136us/step - loss: 1.1214 - acc: 0.5679 - val_loss: 1.0667 - val_acc: 0.5890

Epoch 6/30
3277/3277 [==============================]3277/3277 [==============================] - 0s 127us/step - loss: 1.1103 - acc: 0.5697 - val_loss: 1.0686 - val_acc: 0.5780

Train on 3277 samples, validate on 820 samples
Epoch 1/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 259us/step - loss: 1.1559 - acc: 0.5453 - val_loss: 1.0278 - val_acc: 0.6195

Epoch 2/30
3277/3277 [==============================]3277/3277 [==============================] - 1s 254us/step - loss: 1.1462 - acc: 0.5633 - val_loss: 1.0571 - val_acc: 0.6098

Epoch 3/30
3277/3277 [==============================]3277

In [50]:
    a = 0
    b = 10
    
    data_range = (a,b)
    
    data = conn.query_to_df(QUERY % data_range, 'record')
    
    data = data.abs()
    
    # Check head rows
    print(data.head())
    
    # drop class from data for training
    train_X = data.drop(columns=['class'])
    
    #train_X = MinMaxScaler()
    rows = len(train_X)
    cols = len(train_X.columns)
    
    pre_scl = train_X.values.reshape(rows*cols, 1)
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(pre_scl)
    train_X_res = scaler.transform(train_X)
    
    train_X = train_X_res.reshape(rows, cols)
    
    train_X = pd.DataFrame(train_X)
    
    # Check head rows
    train_X.head()
    
    # convert to binarise 7 classes from 'class' column
    train_y = to_categorical(data['class'], num_classes = 8)
    
    # Check head rows after conversion
    print(train_y[0:5])
    
    print(train_X)

        channel1  channel2  channel3  channel4  channel5  channel6  channel7  \
record                                                                         
1        0.00083   0.00004   0.00031   0.00095   0.00061   0.00026   0.00013   
2        0.00001   0.00003   0.00003   0.00019   0.00007   0.00013   0.00002   
3        0.00001   0.00001   0.00001   0.00006   0.00001   0.00000   0.00001   
4        0.00002   0.00003   0.00004   0.00001   0.00005   0.00005   0.00005   
5        0.00011   0.00002   0.00004   0.00005   0.00013   0.00004   0.00002   

        channel8  class  
record                   
1        0.00002    2.0  
2        0.00000    5.0  
3        0.00000    1.0  
4        0.00005    4.0  
5        0.00010    3.0  
[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]]
          0         1         2         3         4         5         6  \
0  0.747368 -0.915789 -0.347368  1.000000  0.2

In [ ]:
conn.close()

In [20]:
df = pd.DataFrame(np.arange(12).reshape(3,4), columns=['A', 'B', 'C', 'D'])
print(df)
df = df.drop(columns=['B', 'C'])
print(df)

   A  B   C   D
0  0  1   2   3
1  4  5   6   7
2  8  9  10  11
   A   D
0  0   3
1  4   7
2  8  11
